In [1]:
!pip install num2words
!pip install word2number
!pip install evaluate
!pip install wandb

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 5.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c6ba33c133aaee36f0fd85f179904f311950fed181fefc950ff506f581bfbe46
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt
  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5568 sha256=5b4fa786ec49914c0c1667d10442964a7ff621647bd022685dbaf3fcd00c51c2
  Stored in directory: /root/.cache/pip/wheels/cd/ef/ae/073b491b14d25e2efafcffca9e16b2ee6d114ec5c643ba4f06
Successfully built word2number
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [4]:
import json
import os
import pandas as pd
import re
import time
import nltk
import random
from num2words import num2words
from word2number import w2n
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download("stopwords")
from datetime import timedelta
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, pipeline
import torch
import numpy as np
import evaluate
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import wandb
wandb.init(mode = "disabled")

**Збір та підготовка даних**

1. Зібрати переписку всіх чатів від клієнта

2. Підготувати stopwords для вилучення непотрібних слів(це зроблено для того, щоб модель не завчалась на не основих словах, таких як: "привіт", "так", "ок", "ні", "грн", "тис" і т.д)

3. Далі відбираємо топ 1000 популярних слів з усіх чатів за допомогою word_tokenize(це для того щоб навчити модель на різних наборах зустрічаємих слів)

4. Далі робимо дві функції:
  * Функція generate_random_sum(), яка буде генерувати випадкові суми по паттеранах, які ми визначали для вилучення. Наприклад: 100к, 100тис.грн, сто тисяч, 100тыс. 1к гривень, одинадцять тисяч шістот гривень, двести тысяч долларов.
  * Та функція generate_sentence(), яка буде генерувати у випадковому порядку різний набір речень зі пункту 3(топ 1000 популярних слів)
  * Об'єднати дві функції, щоб на виході було - рандомне згенероване речення + рандомна сума по паттернах.

In [6]:
#Це датафрейм чатів зі зверненями від клієнта(для прикладу виведу структуру та звернення по 10 чатах у зв'язку з конф. даними)
#Загалом весь датасет містив ~1100000 рядків
df_result = pd.read_excel('./data/test_lessons_14.xlsx', index_col=False)
df_result

,conversation_id,content,rank_desc
0,1,Зменшити кредитний ліміт,1
1,1,Як зменшити кредитний ліміт,2
2,1,Добрий вечір,3
3,1,Як зменшити кредитний ліміт,4
4,1,"Добре , дякую , звернусь зранку",5
...,...,...,...
61,10,"Подавала, написало що відхилено",2
62,10,Дуже дякую,3
63,10,Дякую,4
64,10,"Так, користуюсь",5


**Так як код внизу буде не повноцінно працювати без всього набору даних, я підвантажу вже готовий датасет після обробки - змінна prepared_data**

In [16]:
russian_stopwords = stopwords.words("russian")
stopwords_ua = pd.read_csv("./data/stopwords_ua.txt", header=None, names=['stopwords'])
ukraine_stopwords = list(stopwords_ua.stopwords)
new_stopwords_ru_ua = russian_stopwords + ukraine_stopwords

In [ ]:
from collections import Counter
word_counter = Counter()


for text in df_result['content']:

    tokens = word_tokenize(text.lower())  # Перетворення на нижній регістр
    tokens = [word for word in tokens if word.isalpha()]  # Залишаємо тільки слова
    tokens = [i for i in tokens if i not in new_stopwords_ru_ua]
    word_counter.update(tokens)  # Оновлюємо лічильник

word_count_df = pd.DataFrame(word_counter.items(), columns=['word', 'count'])
word_count_df = word_count_df.sort_values(by='count', ascending=False).reset_index(drop=True)

word_count_df_new = word_count_df[:1000]
word_count_df_new

In [ ]:
def generate_random_sum():
    # Набори для форматів
    numeric_formats = ["{:.0f}", "{:.2f}", "{:.3f}"]
    text_formats = ["{amount_text}", "{amount_text} гривень", "{amount_text} грн", "{amount_text} гривен"]
    measures = [""," ","грн", " грн", "гривень", "гривен", "$", "₴", "€"]
    sizing = [""," ", "к", "тыс", "тис"]

    # Генерація випадкової суми
    amount = random.randint(1, 100) * 1000  # Генеруємо суми кратні 1000

    # Випадково вибираємо форматування з додатковими нулями чи без
    add_full_format = random.choice([True, False])

    # Форматування суми числово
    numeric_format = random.choice(numeric_formats)
    measure = random.choice(measures)
    size = random.choice(sizing)

    if size in ["тыс", "тис", "к"]:
        display_amount = amount / 1000  # Для відображення скороченої форми
        numeric_result = "{:.0f}".format(display_amount) + size + " " + measure.strip()
    else:
        if add_full_format and numeric_format == "{:.2f}":
            numeric_result = numeric_format.format(amount) + " " + measure.strip()
        else:
            numeric_result = "{:.0f}".format(amount) + " " + measure.strip()

    # Форматування суми словами
    lang = random.choice(["uk", "ru"])
    amount_text = num2words(amount, lang=lang, to="cardinal")
    text_format = random.choice(text_formats)
    text_result = text_format.format(amount_text=amount_text)

    # Рандомний вибір між числовим і текстовим форматом
    return random.choices([numeric_result.strip(), text_result.strip()], weights=[9, 1])[0]

# Функція для вибору слова з урахуванням частоти
def weighted_choice(word_df, used_words):
    available_words = word_df[~word_df['word'].isin(used_words)]  # Уникаємо повторень
    if available_words.empty:
        return None  # Якщо всі слова використані
    words = available_words['word'].tolist()
    weights = available_words['count'].tolist()
    return random.choices(words, weights=weights, k=1)[0]

# Функція для генерації речення
def generate_sentence(word_df, random_amount):
    sentence_length = random.randint(3, 10)  # Випадкова довжина речення
    used_words = []
    words = []

    for _ in range(sentence_length - 1):
        word = weighted_choice(word_df, used_words)
        if word:
            words.append(word)
            used_words.append(word)

    # Додаємо суму в рандомну позицію
    random_position = random.randint(0, len(words))
    words.insert(random_position, random_amount)

    sentence = ' '.join(words)
    return sentence

In [ ]:
# prepared_data = []

# for _ in range(1000): #К-сть рандомно згенерованих речень

#     random_amount = generate_random_sum()
#     sentence = generate_sentence(word_count_df_new,random_amount)

#     prepared_data.append({
#             "prompt": sentence,
#             "question": "Яка сума тут вказана?",
#             "response": random_amount
#             })

In [7]:
with open('./data/test.json', 'r', encoding='utf=8') as f:
    data = json.load(f)


prepared_data = []
for item in data:
    prepared_data.append({
        "prompt": item["prompt"],
        "question": item["question"],
        "response": item["response"]
    })

In [8]:
prepared_data

[{'prompt': 'добрий шістдесят шість тисяч грн зразу бачу',
  'question': 'Яка сума тут вказана?',
  'response': 'шістдесят шість тисяч грн'},
 {'prompt': '97к картки користуюся картці скористатися частинами могу знов ліміт',
  'question': 'Яка сума тут вказана?',
  'response': '97к'},
 {'prompt': 'кредитний перекази збільшити 69к підвищити',
  'question': 'Яка сума тут вказана?',
  'response': '69к'},
 {'prompt': '16тис $ карті кредитним нема заблоковано день відповідь хочу зрозуміло',
  'question': 'Яка сума тут вказана?',
  'response': '16тис $'},
 {'prompt': 'смс 50тыс добрий',
  'question': 'Яка сума тут вказана?',
  'response': '50тыс'},
 {'prompt': 'перекази 54000.00 гривень чекаю',
  'question': 'Яка сума тут вказана?',
  'response': '54000.00 гривень'},
 {'prompt': '59000 $ вийде хотіла робив добрий наразі ласка зняття',
  'question': 'Яка сума тут вказана?',
  'response': '59000 $'},
 {'prompt': 'ліміт пише 97000.00 хочу ласка',
  'question': 'Яка сума тут вказана?',
  'respon

**Вибір моделі GenAI**

Для fine-tuning моделі вирішив обрати вже pre-trained модель навчену на українській мові - ukr-models/xlm-roberta-base-uk

In [9]:
tokenizer = AutoTokenizer.from_pretrained("ukr-models/xlm-roberta-base-uk")
model = AutoModelForQuestionAnswering.from_pretrained("ukr-models/xlm-roberta-base-uk")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/854k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at ukr-models/xlm-roberta-base-uk and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Створення Dataset
train_data, val_data = train_test_split(prepared_data, test_size=0.3, random_state=42)

train_dataset = Dataset.from_dict({key: [d[key] for d in train_data] for key in train_data[0]})
val_dataset = Dataset.from_dict({key: [d[key] for d in val_data] for key in val_data[0]})

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

In [11]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['prompt', 'question', 'response'],
        num_rows: 700
    })
    validation: Dataset({
        features: ['prompt', 'question', 'response'],
        num_rows: 300
    })
})

In [12]:
#Препроцесінг та токенізація
def preprocess_function(examples):
    #print(examples)
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        # questions,
        # examples["prompt"],
        # truncation=True,
        # return_offsets_mapping=True,
        # padding=True,
        questions,
        examples["prompt"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["response"]
    contexts = examples["prompt"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]

        if answer is None:
            print(questions[i], answer, contexts[i])
        start_char = contexts[i].find(answer)
        end_char = start_char + len(answer)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


tokenized_train = dataset_dict['train'].map(preprocess_function, batched = True)
tokenized_val = dataset_dict['validation'].map(preprocess_function, batched = True)

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [13]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

**Тренування та оптимізація моделі**

In [14]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1).flatten()
    labels = np.concatenate(labels) if isinstance(labels, tuple) else labels
    return metric.compute(predictions=predictions, references=labels)

In [15]:
training_args = TrainingArguments(
    output_dir=".data/model_result/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=3,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-2d26977d2024>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.010955,0.991667
2,No log,0.005660,0.998333
3,No log,0.001157,1.000000
4,0.295600,0.000178,1.000000
5,0.295600,0.000085,1.000000


TrainOutput(global_step=700, training_loss=0.2112305979749986, metrics={'train_runtime': 405.2614, 'train_samples_per_second': 8.636, 'train_steps_per_second': 1.727, 'total_flos': 685903986432000.0, 'train_loss': 0.2112305979749986, 'epoch': 5.0})

In [17]:
trainer.save_model("./model_result/ukr-models-xlm-roberta-base-uk-amount")

**Оцінка результатів та поліпшення**

In [21]:
dir_model = './model_result/ukr-models-xlm-roberta-base-uk-amount'

tokenizer = AutoTokenizer.from_pretrained(dir_model)
model = AutoModelForQuestionAnswering.from_pretrained(dir_model)

qa_model = pipeline("question-answering", model = model, tokenizer = tokenizer)

Device set to use cuda:0


In [24]:
question = "Яка сума тут вказана?"

context = ["Як мені підняти ліміт до 100к?",
           "Хочу збільшити лміт до сто тисяч",
           "Добрий день! прошу зменшити ліміт до 100тис.грн",
           "привіт. збільшіть ліміт до 1к",
           "привіт!, як справи?",
           "де тут бібліотека?"]

for i in context:
  print(i)
  print(qa_model(question = question, context = i.lower()))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Як мені підняти ліміт до 100к?
{'score': 0.9998927116394043, 'start': 24, 'end': 30, 'answer': ' 100к?'}
Хочу збільшити лміт до сто тисяч
{'score': 0.9924783706665039, 'start': 22, 'end': 32, 'answer': ' сто тисяч'}
Добрий день! прошу зменшити ліміт до 100тис.грн
{'score': 0.9995762705802917, 'start': 36, 'end': 47, 'answer': ' 100тис.грн'}
привіт. збільшіть ліміт до 1к
{'score': 0.9979227185249329, 'start': 26, 'end': 29, 'answer': ' 1к'}
привіт!, як справи?
{'score': 0.1057034283876419, 'start': 11, 'end': 19, 'answer': ' справи?'}
де тут бібліотека?
{'score': 0.1489204317331314, 'start': 6, 'end': 18, 'answer': ' бібліотека?'}
